In [ ]:
%%python

from typing import Sequence
import random
import array

from matplotlib import pyplot as plt
from timeit import timeit

class Affine:
    def __init__(self, w11, w12, w21, w22, b1, b2):
        self.w11 = w11
        self.w12 = w12
        self.w21 = w21
        self.w22 = w22
        self.b1 = b1
        self.b2 = b2
    def __call__(self, x, y):
        return (
            self.w11 * x + self.w12 * y + self.b1,
            self.w21 * x + self.w22 * y + self.b2
        )

def pygoma(N):

    aff1 = Affine(
        0., 0.,
        0., 0.16,
        0., 0.,
    )

    aff2 = Affine(
        0.85, 0.04,
        -0.04, 0.85,
        0., 1.6
    )

    aff3 = Affine(
        0.2, -0.26,
        0.23, 0.22,
        0., 1.6,
    )

    aff4 = Affine(
        -0.15, 0.28,
        0.26, 0.24,
        0., 0.44
    )

    xs = array.array("d", [1. for _ in range(N)])
    ys = array.array("d", [1. for _ in range(N)])
    x = 0.
    y = 0.
    for i in range(N):
        aff = None
        r = random.random()
        if r < 0.01:
            aff = aff1
        elif 0.01 <= r < 0.86:
            aff = aff2
        elif 0.86 <= r < 0.93:
            aff = aff3
        else:
            aff = aff4
        x, y = aff(x, y)
        xs[i] = x
        ys[i] = y
    return xs, ys

xs, ys = pygoma(N = 100000)

plt.scatter(xs, ys)

pymsecs = 1000 * timeit(lambda: pygoma(N=100000), number=10)/10
print(pymsecs, "ms")

plt.show()

In [3]:
from Benchmark import Benchmark
from Vector import DynamicVector
from Random import random_f64
from Pointer import DTypePointer
from DType import DType

let plt = Python.import_module("matplotlib.pyplot")
let np = Python.import_module("numpy")

@register_passable("trivial")
struct Point:
    var x: F64
    var y: F64
    
    fn __init__(x: F64, y: F64) -> Self:
        return Self {x: x, y: y}
        
@register_passable("trivial")
struct Affine:
    var w11: F64
    var w12: F64
    var w21: F64
    var w22: F64
    var b1: F64
    var b2: F64

    fn __init__(w11: F64, w12: F64, w21: F64, w22: F64, b1: F64, b2: F64) -> Self:
        return Self {
            w11 : w11,
            w12 : w12,
            w21 : w21,
            w22 : w22,
            b1 : b1,
            b2 : b2,
        }
        
@register_passable("trivial")
struct MyResult:
    var x: DTypePointer[DType.f64]
    var y: DTypePointer[DType.f64]
    
    fn __init__(x: DTypePointer[DType.f64], y: DTypePointer[DType.f64]) -> Self:
        return Self {x: x, y: y}

fn mojogoma(N: Int) -> MyResult:
    let aff1 = Affine(
        0., 0.,
        0., 0.16,
        0., 0.,
    )

    let aff2 = Affine(
        0.85, 0.04,
        -0.04, 0.85,
        0., 1.6
    )

    let aff3 = Affine(
        0.2, -0.26,
        0.23, 0.22,
        0., 1.6,
    )

    let aff4 = Affine(
        -0.15, 0.28,
        0.26, 0.24,
        0., 0.44
    )

    let xs = DTypePointer[DType.f64].alloc(N)
    let ys = DTypePointer[DType.f64].alloc(N)
    var x: F64 = F64(0.)
    var y: F64 = F64(0.)
    for i in range(N):
        let r = random_f64()
        let aff: Affine
        if r < F64(0.01):
            aff = aff1
        elif F64(0.01) <= r < F64(0.86):
            aff = aff2
        elif F64(0.86) <= r < F64(0.93):
            aff = aff3
        else:
            aff = aff4
        let x_next = aff.w11 * x + aff.w12 * y + aff.b1
        let y_next = aff.w21 * x + aff.w22 * y + aff.b2
        
        xs.store(i, x)
        ys.store(i, y)
        x = x_next
        y = y_next
    return MyResult(xs, ys)

def gomabench(N: Int):
    @parameter
    fn test_fn():
        _ = mojogoma(N)
    msecs = F64(Benchmark().run[test_fn]())/1_000_000
    print(msecs, "ms")
    
let N = 100000
print(gomabench(N))
let r = mojogoma(N)
let pyx = np.zeros(N, np.float64)
let pyy = np.zeros(N, np.float64)

for i in range(N):
    pyx.itemset(i, r.x.load(i))
    pyy.itemset(i, r.y.load(i))
    
plt.scatter(pyx, pyy)
plt.show()

SyntaxError: invalid syntax (1622646226.py, line 7)